# Ejecta model

TARDIS requires a model of the homologously expanding ejecta in order to run a simulation. The ejecta is modeled as concentric 1 dimensional radial cells. Each cell contins information the velocity structure, abundances, density, and a radiation field.
#This information is stored in TARDIS [config](link). 
The information of these properties can be stored as classes and all of them inherit from a BaseProperty to facilitate the interaction. BaseProperty handles the initailization of the property's time and initial time (time_0) with which most properties build on.             
##  Model grid
The first thing needed in order to have a useful ejecta model is a discreet space grid. 
What defines the spatial grid in TARDIS is the velocity grid. This dependance on velocity also entails a dependance with time.
### Velocity Class
For a basic grid inner and outer velocities are needed. Each concentric cell has an inner and outer velocity, and inner_velocity and outer_velocity are arrays composed of these velocities.
The velocity class stores velocities and velocities-related quantities (such as cell radius and volume). To start a Velocity class we need `inner_velocity` and an `outer_velocity` arrays, optional arguments are `time_0` which fallback to 0 seconds and `time` which is recomended to have singificant values for radii and volume.


In [1]:
from tardis.model._generic_model import *

/home/jordi/conda/envs/tardis/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.data is not yet QA compliant.
  return f(*args, **kwds)


In [ ]:
velocity_inner = [i for i in range(9000, 13000, 500)] * u.km / u.s
velocity_outer = [i for i in range(9500, 13500, 500)] * u.km / u.s
print(velocity_inner,'\n',velocity_outer)

In [ ]:
velocity = Velocity(velocity_inner,velocity_outer)


Quantities are converted to CGS and can be easily accessed:

In [ ]:
velocity.middle

By not providing a time for the class some quantities don't make sense:

In [ ]:
velocity.inner_radius

`time` can be modified:

In [ ]:
velocity.time = 1 * u.d
velocity.time

Quantities stored in Velocity are:
 - Velocity.inner 
 - Velocity.outer
 - Velocity.middle
 - Velocity.inner_radius
 - Velocity.middle_radius
 - Velocity.outer_radius
 - Velocity.volume
 - Velocity.time_0
 - Velocity.time
 - Velocity.number_of_cells

The number of cells is defined by the length of the input velocity arrays

In [ ]:
velocity.number_of_cells

### Density
Once the grid is defined, more properties can be added to the structure. 
The Density class can store mass density and electron density(optional).
Initial mass density is stored as `mass_0`.

In [6]:
mass_density_array = [
            6.2419e-10,
            6.1502e-10,
            6.0492e-10,
            5.9475e-10,
            5.8468e-10,
            5.7473e-10,
            5.6493e-10,
            5.5529e-10,
        ] * u.g / u.cm**3
density = Density(mass_density_array, time = 1 * u.d)

Initial mass density is stored as `mass_0`.

In [5]:
density.mass_0

<Quantity [6.2419e-10, 6.1502e-10, 6.0492e-10, 5.9475e-10, 5.8468e-10,
           5.7473e-10, 5.6493e-10, 5.5529e-10] g / cm3>

The ejecta is homologouly expanding, and therefore the value of the density will decrease proportionally to (time/time_0)**-3. This again highlights the importance of providing a `time` for the class.

In [9]:
density.mass

<Quantity [9.67777630e-25, 9.53559970e-25, 9.37900389e-25, 9.22132276e-25,
           9.06519208e-25, 8.91092195e-25, 8.75897750e-25, 8.60951377e-25] g / cm3>

Quantities stored in Density are:
 - Density.mass_0
 - Density.mass
 - Density.electron
 - Density.time_0
 - Density.time
 - Density.number_of_cells

### Abundance
Explain
Example
### Radiation Field
Explain
Example
## Generic[Ejecta]Model
A more flexible implementation of the TARDIS ejecta model is the Generic[Ejecta]Model. 
What does Generic model is and what it does with data.

Explain what you can do here but not in TARDIS
Start model and different times using above properties 
Explain that more properties can be added as long as they use the basxi structure of BaseProperty
## TARDIS Ejecta Model
Explain what happens with v_boundary and that it can't change.